# 벨만-포드 (Bellman-Ford)

`-` 정점이 $V$개, 간선이 $E$개인 음의 가중치가 존재하는 그래프에서 임의의 정점과 나머지 정점간의 최단 경로를 $O(VE)$에 찾는 알고리즘 

## 웜홀

- 문제 출처: [1865번](https://www.acmicpc.net/problem/1865)

`-` 플로이드-워셜 알고리즘을 사용하여 문제를 해결할 수 있다

`-` 플로이드-워셜 알고리즘을 적용 후 임의의 노드 $u$에 대해 $u\to u$의 가중치가 음수라면 음수 사이클이 존재하는 것이다

`-` 이는 웜홀을 통해 시간이 되돌아 가는 경우가 있음을 뜻한다

`-` 플로이드-워셜 알고리즘에서 $u\to u$의 가중치는 $0$으로 초기화하고 시작한다

`-` 그런데 이것이 음수가 됐다는 것은 $u\to k + k \to u$가 $0$보다 작다는 뜻이다

`-` 즉, $u$에서 $u$로 가는 음수 사이클이 존재한다는 것이다

`-` 그런데 안타깝게도 91%에서 시간 초과가 발생한다

`-` 다른 방법으로 알고리즘 시간에 배운 [벨만-포드](https://en.wikipedia.org/wiki/Bellman%E2%80%93Ford_algorithm) 알고리즘을 사용했다

`-` 음수 간선이 있어도 사용 가능하며 만약 알고리즘이 끝난 후에도 경로 갱신이 가능하다면 음수 사이클이 내부에 존재한다는 것이다

`-` 벨만-포드 알고리즘의 시간 복잡도는 $O(VE)$이다

`-` 시작 노드와의 연결 유무 상관 없이 모든 간선에 대해 최단 경로를 업데이트 하므로 어느 하나의 노드에 대해서 벨만-포드 알고리즘을 한 번만 적용하면 된다

In [49]:
def has_negative_cycle(start, adj_list):
    # 벨만-포드 알고리즘
    dist = [INF for _ in range(N + 1)]
    dist[start] = 0
    for _ in range(N + 1):  # 더 최적화 하려면 노드 개수를 직접 세면 된다 (중복 노드가 입력될 수 있다)
        for (s, e), t in adj_list.items():
            if dist[s] + t < dist[e]:
                dist[e] = dist[s] + t
    # 음수 사이클 확인
    answer = "NO"
    for (s, e), t in adj_list.items():
        if dist[s] + t < dist[e]:
            answer = "YES"
            break
    return answer


def solve_testcase():
    global N, INF
    N, M, W = map(int, input().split())
    INF = 1e9  # float("inf") 사용하면 안된다, float("inf")와의 상수 연산은 float("inf")이므로 갱신이 안됨
    adj_list = {}
    for _ in range(M):
        s, e, t = map(int, input().split())
        adj_list[(s, e)] = t if (s, e) not in adj_list else min(adj_list[(s, e)], t)
        adj_list[(e, s)] = t if (e, s) not in adj_list else min(adj_list[(e, s)], t)
    for _ in range(W):
        s, e, t = map(int, input().split())
        adj_list[(s, e)] = -t
    print(has_negative_cycle(1, adj_list))


def solution():
    TC = int(input())
    for _ in range(TC):
        solve_testcase()


solution()

# input
# 2
# 3 3 1
# 1 2 2
# 1 3 4
# 2 3 1
# 3 1 3
# 3 2 1
# 1 2 3
# 2 3 4
# 3 1 8

 2
 3 3 1
 1 2 2
 1 3 4
 2 3 1
 3 1 3


NO


 3 2 1
 1 2 3
 2 3 4
 3 1 8


YES


## 타임머신

- 문제 출처: [백준 11657번](https://www.acmicpc.net/problem/11657)

`-` 음수 간선이 있는 그래프에서 최단 경로를 찾는 문제이다

`-` 벨만-포드 알고리즘 사용해 문제를 해결할 수 있다

`-` 정점의 개수를 $V$라 할 때 $V-1$번의 갱신 후에도 최단 경로가 갱신되면 음수 사이클이 존재하는 것이다

`-` 중요한 건 그래프 내에 음수 사이클이 존재하냐가 아니라 $1$번과 연결된 음수 사이클이 존재하냐는 것이다

`-` 즉, $1$번과 고립된 영역에 있는 음수 사이클은 문제가 되지 않는다

`-` 이를 위해 최단 경로 추정치 배열을 초기화할 때 `float("inf")`를 사용했다

`-` 만약 순수하게 음수 사이클 여부가 궁금하다면 매우 큰 상수로 초기화하면 된다

In [16]:
def bellman_ford(start, graph, weight):
    dist = [INF for _ in range(N + 1)]  # 출발지부터 다른 지점까지의 최단 경로 추정치
    dist[start] = 0
    for _ in range(N - 1):
        for u, v in graph:
            if dist[u] + weight[(u, v)] < dist[v]:
                dist[v] = dist[u] + weight[(u, v)]
    return dist


def check_negative_cycle(dist, graph, weight):
    for u, v in graph:
        if dist[u] + weight[(u, v)] < dist[v]:
            return True
    return False


def solution():
    global N, INF
    N, M = map(int, input().split())
    INF = float("inf")
    START = 1
    graph = []
    weight = {}
    for _ in range(M):
        A, B, C = map(int, input().split())
        if (A, B) not in weight:
            graph.append((A, B))
            weight[(A, B)] = C
        else:
            weight[(A, B)] = min(C, weight[(A, B)])
    dist = bellman_ford(START, graph, weight)
    has_negative_cycle = check_negative_cycle(dist, graph, weight)
    if has_negative_cycle:
        print(-1)
        return
    for i in range(2, N + 1):
        if dist[i] == INF:
            print(-1)
        else:
            print(dist[i])


solution()

# input
# 3 4
# 1 2 4
# 1 3 3
# 2 3 -1
# 3 1 -2

 3 4
 1 2 4
 1 3 3
 2 3 -1
 3 1 -2


4
3
